In [ ]:
"""using nographs + graphviz for program environment inspection"""
from decorator import decorator
from functools import wraps, update_wrapper
from ctypes    import cast, py_object
from graphviz  import Digraph
import nographs as nog
# http://magjac.com/graphviz-visual-editor/

In [ ]:
node_style =dict( style='filled', shape='box', 
                                align='left',
                                fontsize='12',
                                ranksep='0.01',
                                height='0.02',width='0.04',)
gr_style = dict( linelength="16",rankdir='LR',size='32,16',
                # concentrate='true'
               )#landscape="True",splines='curved' ,layout="neato"

def adapter_graphviz( wrapped ):
    """translation to GraphViz visitor"""
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
    @wraps( wrapped )
    def wrapper_bg( vert, _trav ):
        dot.node( str(vert), str(vert) )
        for result in wrapped(vert, _trav):
            dot.edge( str(vert), str(result), )# label=f"{str(x)}-{str(result)}",
            yield result
    wrapper_bg.dot = dot
    return wrapper_bg


In [ ]:
def graph_inspect( obj, forward ):
    trav_forward = nog.TraversalBreadthFirst( forward )
    trav_forward.start_from( obj, build_paths=True )
    for _, _ in zip(trav_forward, range(50)):
        pass
    graph_inspect.trav_forward = trav_forward
    return forward

In [ ]:
@decorator
def filter_boring( wrapped, vert, _trav ):
    stop_list = {'__dir__'}
    def filter_func( item ):
        nam = getattr( item, '__name__', 'NoName' )
        print( f"{item}---{type(item)}---{nam=}---{nam in stop_list}" )
        return type(item).__name__ not in stop_list
    """Universal decorator for filtering"""
    yield from filter( filter_func, wrapped( vert, _trav ) )

@decorator
def concentrate_edges( wrapped, vert, _trav ):
    """Universal decorator for edge concentration (GraphViz term)"""
    yield from set(wrapped( vert, _trav ))

In [ ]:
class DressCovariant:
    # pylint: disable=no-member   #(__wrapped__)
    """morpher for node names base"""
    def __init__(self, wrapped):
        update_wrapper(self, wrapped )
    @staticmethod
    def dress( vert,):
        return str(vert)[:30]
    def __call__(self, vert, _trav):
        yield from map( self.dress, self.__wrapped__( vert , _trav ) )

In [ ]:
@adapter_graphviz
@concentrate_edges
@DressCovariant
@filter_boring
def all_attr_2(var, _trav):
    for attr_name, _ in zip(dir(var), range(20)):
        try:
            attr = getattr(var, attr_name)
            if hasattr( attr, "__call__"):
                try:
                    attr = attr()
                except Exception as ex:
                    attr = str(ex)[:50]
            if attr is not None: #hasattr( attr, "__hash__"):
                yield attr
        except:
            #print(f"!!!{var}-{attr_name}")
            continue
graph_inspect( 10, all_attr_2 ).dot

In [ ]:
# class DressCovariant:
#     # pylint: disable=no-member   #(__wrapped__)
#     """morpher for node names base"""
#     def __init__(self, wrapped):
#         update_wrapper(self, wrapped )
#     @staticmethod
#     def dress( vert,):
#         return vert % 10
#     def __call__(self, vert, _trav):
#         yield from map( self.dress, self.__wrapped__(              vert , _trav ) )


# @adapter_graphviz
# @concentrate_edges
# @DressCovariant
# def covar_inside(vert, _):
#     for attr_name, _ in zip(dir(var), range(50)):
#         try:
#             attr = getattr(var, attr_name)
#             if attr is not None: #hasattr( attr, "__hash__"):
#                 yield attr
#         except:
#             continue

# graph_inspect( 0, covar_inside ).dot

In [ ]:
type(graph_inspect.trav_forward)

In [ ]:
dir(graph_inspect.trav_forward)

In [ ]:
graph_inspect( graph_inspect.trav_forward ).dot